**This data set contains details of a bank's customers and the target variable is a binary variable reflecting the fact whether the customer left the bank (closed his account) or he continues to be a customer.**

In [1]:
import pandas as pd 
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import  StandardScaler,LabelEncoder
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data.shape

(10000, 14)

In [4]:
## Data Preprocessing
##Drope irrelevant Features
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


**Two Categorical Features are present in this dataset**

In [6]:
#Encode Categorical features
label_encod_gender=LabelEncoder()
data['Gender']=label_encod_gender.fit_transform(data['Gender'])

In [7]:
data['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [8]:
#One hot encode Geography column
from sklearn.preprocessing import  OneHotEncoder
onehot_encode_geography=OneHotEncoder()
geo_encoder=onehot_encode_geography.fit_transform(data[['Geography']]).toarray()
geo_encoder                         

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [9]:
encoded_columns=onehot_encode_geography.get_feature_names_out(['Geography'])
encoded_columns

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geography_encoad_df=pd.DataFrame(geo_encoder,columns=encoded_columns)

In [11]:
#combine att the columns 
data=pd.concat([data.drop('Geography',axis=1),geography_encoad_df],axis=1)

In [12]:
## save the encoder 
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encod_gender,file)


with open('onehot_encode_geography.pkl','wb') as file :
    pickle.dump(onehot_encode_geography,file)    

In [13]:
# Independent and dependent Features  
X=data.drop(['Exited'],axis=1)
y=data['Exited']

In [14]:
#Split the data into train and test set
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [15]:
#scaled the data 
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [16]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [17]:
#save the scaler 
with open ('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

### Implimentation of ANN

In [18]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense #Dense is used for create hidden neurons.
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime


In [19]:
X_train.shape[1],

(12,)

In [20]:
# ANN model build 
model=Sequential( 
    [
        Dense(64,activation='relu',input_shape=(X_train.shape[1],)),# HL1 connected with input layer
        Dense(32,activation='relu'),#HL2
        Dense(1,activation='sigmoid')#Output layer
    ]
)

c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)#Optimizer

loss=tensorflow.keras.losses.BinaryCrossentropy() #Loss Function


In [23]:
#compile the model 
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [25]:
# setup the Tensorboard 
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [26]:
#setup Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [27]:
# Model Tranning 
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8010 - loss: 0.4423 - val_accuracy: 0.8535 - val_loss: 0.3538
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8453 - loss: 0.3674 - val_accuracy: 0.8570 - val_loss: 0.3496
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8587 - loss: 0.3565 - val_accuracy: 0.8670 - val_loss: 0.3398
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8577 - loss: 0.3457 - val_accuracy: 0.8580 - val_loss: 0.3432
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8601 - loss: 0.3299 - val_accuracy: 0.8580 - val_loss: 0.3406
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8570 - loss: 0.3429 - val_accuracy: 0.8630 - val_loss: 0.3406
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8634 - loss: 0.3310 - val_accuracy: 0.8600 - val_loss: 0.3434
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8622 - loss: 0.3314 - val_accu

In [28]:
model.save('model.h5')

In [29]:
# Load TensorBoard Extention 
%load_ext tensorboard

In [35]:
%tensorboard --logdir logs/fit20240904-131657


Reusing TensorBoard on port 6008 (pid 14460), started 0:00:02 ago. (Use '!kill 14460' to kill it.)